In [1]:
import sys
sys.path.append('../../../')

In [2]:
import pandas as pd
import numpy as np

from modules import machine_learning_utils as mlu

pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)

# 1.0 Data retrieval

In [3]:
df = pd.read_pickle("../../../data_lake/output_prod/df_under.pkl")

# 2.0 Data Partitioning

In [6]:
df['isFraud'].value_counts()

0    132915
1     11318
Name: isFraud, dtype: int64

## 2.1 Train, validation, test split

In [7]:
train, val, test = mlu.partitioning(df,0.7,0.15,'isFraud')

Train shape:  (100963, 41)
Train %: 
 0    0.920496
1    0.079504
Name: isFraud, dtype: float64
Val %: 
 0    0.923319
1    0.076681
Name: isFraud, dtype: float64
Test %: 
 0    0.924567
1    0.075433
Name: isFraud, dtype: float64


## 2.2 Adversarial validation between val and test set

In [8]:
drop_cols = ['TransactionID','customer_id']

In [9]:
mlu.adversarial_validation(val,test,drop_cols)

/src/notebooks/prod_version/2_data_preparation/../../../modules/machine_learning_utils.py:75: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



No distribution shift, OK! AUC is:  0.5046975542178611


## 2.3 Class imbalance

In [10]:
final_train = mlu.class_imbalance(train,'isFraud')

1    8027
0    8027
Name: isFraud, dtype: int64
Train dataset shape:  (16054, 41)


/src/notebooks/prod_version/2_data_preparation/../../../modules/machine_learning_utils.py:113: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



# 3.0 Output

In [11]:
final_train.to_pickle('../../../data_lake/output_prod/train.pkl')
val.to_pickle('../../../data_lake/output_prod/val.pkl')
test.to_pickle('../../../data_lake/output_prod/test.pkl')